In [ ]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.7 MB/s eta 0:00:00


In [ ]:
!uv pip install ultralytics opencv-python-headless scikit-learn torch torchvision torchaudio -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.metrics import precision_score, recall_score
import torch
from scipy.spatial import distance

In [16]:
# Чтение YOLO-аннотаций
def read_yolo_annotations(file_path, img_width=1094, img_height=540):
    stones = []
    house = None
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            norm_center_x = float(parts[1])
            norm_center_y = float(parts[2])
            norm_width = float(parts[3])
            norm_height = float(parts[4])
            if class_id in [0, 1]:  # Желтые (0) или красные (1) камни
                pixel_x, pixel_y = norm_to_pixel_coords(norm_center_x, norm_center_y, img_width, img_height)
                stones.append({
                    'class_id': class_id,
                    'pixel_x': pixel_x,
                    'pixel_y': pixel_y,
                    'norm_width': norm_width,
                    'norm_height': norm_height
                })
            elif class_id == 2:  # Дом
                pixel_x, pixel_y = norm_to_pixel_coords(norm_center_x, norm_center_y, img_width, img_height)
                house = {
                    'pixel_x': pixel_x,
                    'pixel_y': pixel_y,
                    'norm_width': norm_width,
                    'norm_height': norm_height
                }
    return stones, house

# Перевод нормализованных координат в пиксельные
def norm_to_pixel_coords(norm_x, norm_y, img_width, img_height):
    pixel_x = norm_x * img_width
    pixel_y = norm_y * img_height
    return pixel_x, pixel_y

# Вычисление матрицы гомографии
def compute_homography(house, img_width=1094, img_height=540):
    norm_width = house['norm_width']
    norm_height = house['norm_height']
    center_x = house['pixel_x']
    center_y = house['pixel_y']
    half_width = (norm_width * img_width) / 2
    half_height = (norm_height * img_height) / 2

    pixel_points = np.array([
        [center_x - half_width, center_y - half_height],  # Верхний левый
        [center_x + half_width, center_y - half_height],  # Верхний правый
        [center_x + half_width, center_y + half_height],  # Нижний правый
        [center_x - half_width, center_y + half_height]   # Нижний левый
    ], dtype=np.float32)

    house_radius_cm = 182.9
    real_points = np.array([
        [-house_radius_cm, -house_radius_cm],
        [house_radius_cm, -house_radius_cm],
        [house_radius_cm, house_radius_cm],
        [-house_radius_cm, house_radius_cm]
    ], dtype=np.float32)

    H, _ = cv2.findHomography(pixel_points, real_points)
    return H

# Перевод пиксельных координат в физические (см)
def pixel_to_real_coords(pixel_points, H):
    pixel_points = np.array(pixel_points, dtype=np.float32)
    real_points = cv2.perspectiveTransform(pixel_points[None, :, :], H)
    return real_points[0]

# Подсчёт камней в доме и на линии хог
def count_stones_in_house_and_hog(real_coords, house_radius_cm=182.9, hog_line_y_cm=640):
    stones_in_house = 0
    stones_on_hog = 0
    for x_cm, y_cm in real_coords:
        distance = np.sqrt(x_cm**2 + y_cm**2)
        if distance <= house_radius_cm:
            stones_in_house += 1
        if y_cm <= hog_line_y_cm:
            stones_on_hog += 1
    return stones_in_house, stones_on_hog



# Обработка видео и детекция
def process_video(video_path, model, annotation_dir, output_video_path, csv_path, img_width=1094, img_height=540):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (img_width, img_height))

    csv_data = []
    frame_id = 0
    stone_id = 0
    H = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Чтение аннотаций для текущего кадра (если есть)
        annotation_file = os.path.join(annotation_dir, f"frame_{frame_id}.txt")
        if os.path.exists(annotation_file):
            _, house = read_yolo_annotations(annotation_file, img_width, img_height)
            if house and H is None:  # Вычисляем гомографию один раз
                H = compute_homography(house, img_width, img_height)
                print(f"Гомография вычислена для frame_{frame_id}: {H}")

        # Детекция
        results = model.predict(frame, conf=0.5, iou=0.5)
        detections = results[0].boxes

        frame_stones = []
        pixel_centers = []
        for box in detections:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf.cpu().numpy()
            cls = int(box.cls.cpu().numpy())
            if cls not in [0, 1]:  # Пропускаем дом
                continue
            team = 'yellow' if cls == 0 else 'red'
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            pixel_centers.append([center_x, center_y])
            frame_stones.append({'team': team, 'center_x': center_x, 'center_y': center_y})

        # Перевод координат в сантиметры
        real_coords = pixel_to_real_coords(pixel_centers, H) if H is not None and pixel_centers else []

        # Подсчёт камней
        stones_in_house, stones_on_hog = count_stones_in_house_and_hog(real_coords) if len(real_coords) > 0 else (0, 0)

        # Визуализация
        for i, stone in enumerate(frame_stones):
            x_cm, y_cm = real_coords[i] if i < len(real_coords) else (0, 0)
            x1, y1 = int(stone['center_x'] - 20), int(stone['center_y'] - 20)
            x2, y2 = int(stone['center_x'] + 20), int(stone['center_y'] + 20)
            color = (0, 255, 255) if stone['team'] == 'yellow' else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.circle(frame, (int(stone['center_x']), int(stone['center_y'])), 5, (255, 255, 255), -1)
            label = f"{stone['team']} ({x_cm:.1f}, {y_cm:.1f})"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Отображение количества камней
        cv2.putText(frame, f"House: {stones_in_house}, Hog: {stones_on_hog}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        out.write(frame)

        # Сохранение в CSV (добавляем пиксельные координаты)
        for i, stone in enumerate(frame_stones):
            x_cm, y_cm = real_coords[i] if i < len(real_coords) else (0, 0)
            csv_data.append([frame_id, stone_id, stone['team'], x_cm, y_cm, stone['center_x'], stone['center_y']])
            stone_id += 1

        frame_id += 1

    cap.release()
    out.release()

    # Сохранение CSV
    df = pd.DataFrame(csv_data, columns=['frame_id', 'stone_id', 'team', 'x_cm', 'y_cm', 'pixel_x', 'pixel_y'])
    df.to_csv(csv_path, index=False)

    return csv_data

# Оценка метрик
def evaluate_metrics(annotation_dir, csv_data, img_width=1094, img_height=540):
    ground_truth = []
    for annotation_file in os.listdir(annotation_dir):
        if not annotation_file.endswith('.txt'):
            continue
        frame_id = int(annotation_file.split('_')[1].split('.')[0])
        stones, house = read_yolo_annotations(os.path.join(annotation_dir, annotation_file), img_width, img_height)
        if house:
            H = compute_homography(house, img_width, img_height)
        for stone in stones:
            # Переводим ground truth в сантиметры
            real_coords = pixel_to_real_coords([[stone['pixel_x'], stone['pixel_y']]], H) if house else [[0, 0]]
            x_cm, y_cm = real_coords[0] if len(real_coords) > 0 else (0, 0)
            ground_truth.append({
                'frame_id': frame_id,
                'team': 'yellow' if stone['class_id'] == 0 else 'red',
                'x_cm': x_cm,
                'y_cm': y_cm,
                'pixel_x': stone['pixel_x'],
                'pixel_y': stone['pixel_y']
            })

    # Precision и Recall
    y_true = [1] * len(ground_truth)
    y_pred = []
    for gt in ground_truth:
        # Проверяем, есть ли предсказание для этого кадра и команды
        matches = [d for d in csv_data if d[0] == gt['frame_id'] and d[2] == gt['team']]
        y_pred.append(1 if matches else 0)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)

    # MAE по координатам (в сантиметрах)
    mae = 0
    count = 0
    for gt in ground_truth:
        matches = [d for d in csv_data if d[0] == gt['frame_id'] and d[2] == gt['team']]
        if matches:
            # Находим ближайшее предсказание по пиксельным координатам
            distances = [
                distance.euclidean([gt['pixel_x'], gt['pixel_y']], [m[5], m[6]])
                for m in matches
            ]
            min_idx = np.argmin(distances) if distances else 0
            if distances:
                mae += np.abs(gt['x_cm'] - matches[min_idx][3]) + np.abs(gt['y_cm'] - matches[min_idx][4])
                count += 1

    mae = mae / count if count > 0 else float('inf')

    return precision, recall, mae

# Генерация отчёта
def generate_report(precision, recall, mae, output_path):
    report = f"""
    # Отчёт по детекции камней в кёрлинге

    ## Метрики
    - Precision: {precision:.3f}
    - Recall: {recall:.3f}
    - MAE по координатам (см): {mae:.3f}

    """
    with open(output_path, 'w') as f:
        f.write(report)
    return report

# Основной запуск
def main():

    # Пути к данным
    annotation_dir = '/content/drive/MyDrive/CV_SBER/1_Энд_выборка_labels'

    video_path = '/content/drive/MyDrive/CV_SBER/1_end.mp4'
    output_video_path = '/content/drive/MyDrive/CV_SBER/Итоги/demo_video.mp4'

    model = YOLO('/content/drive/MyDrive/CV_SBER/curling_yolov8.pt')

    csv_path = '/content/drive/MyDrive/CV_SBER/Итоги/stones_coordinates.csv'
    report_path = '/content/drive/MyDrive/CV_SBER/Итоги/report.md'


    # Обработка видео
    csv_data = process_video(video_path, model, annotation_dir, output_video_path, csv_path)

    # Оценка метрик
    precision, recall, mae = evaluate_metrics(annotation_dir, csv_data)

    # Генерация отчёта
    report = generate_report(precision, recall, mae, report_path)

    print("Обработка завершена. Результаты сохранены в:")
    print(f"- Видео: {output_video_path}")
    print(f"- CSV: {csv_path}")
    print(f"- Отчёт: {report_path}")

if __name__ == '__main__':
    main()


0: 544x1088 (no detections), 13.3ms
Speed: 4.6ms preprocess, 13.3ms inference, 0.8ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.9ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.4ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.9ms preprocess, 9.6ms inference, 0.8ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.4ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.4ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.5ms
Speed: 4.4ms preprocess, 9.5ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 (no detections), 9.6ms
Speed: 4.3ms preprocess, 9.6ms 

<ipython-input-16-1c1cee5876b0>:118: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls = int(box.cls.cpu().numpy())


Выходные данные были обрезаны до нескольких последних строк (5000).
Speed: 5.9ms preprocess, 9.6ms inference, 0.9ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 1 red, 9.6ms
Speed: 5.0ms preprocess, 9.6ms inference, 1.9ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 1 red, 9.6ms
Speed: 5.8ms preprocess, 9.6ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 1 red, 9.6ms
Speed: 5.8ms preprocess, 9.6ms inference, 1.9ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 2 reds, 15.2ms
Speed: 5.6ms preprocess, 15.2ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 2 reds, 12.0ms
Speed: 5.8ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 1 red, 12.0ms
Speed: 5.7ms preprocess, 12.0ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 1088)

0: 544x1088 2 reds, 13.4ms
Speed: 4.9ms preprocess, 13.4ms inference, 7.9ms postprocess 